In [1]:
import jax.numpy as jnp

In [13]:
# PLOTS
import plotly.graph_objects as go


def update_layout_of_graph(fig: go.Figure, title: str = 'Plot') -> go.Figure:
    fig.update_layout(
        width=800,
        height=600,
        autosize=False,
        plot_bgcolor='rgba(0,0,0,0)',
        title=title,

    )
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                      xaxis_title='input values',
                      yaxis_title='output values',
                      legend=dict(yanchor="top",
                                  y=0.9,
                                  xanchor="right",
                                  x=0.95),
                      title={
                          'x': 0.5,
                          'xanchor': 'center'
                      })
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
    return fig


def line_scatter(
        visible: bool = True,
        x_lines: jnp.array = jnp.array([]),
        y_lines: jnp.array = jnp.array([]),
        name_line: str = 'Predicted function',
        showlegend: bool = True,
) -> go.Scatter:
    # Adding the lines
    return go.Scatter(
        visible=visible,
        line=dict(color="blue", width=2),
        x=x_lines,
        y=y_lines,
        name=name_line,
        showlegend=showlegend
    )


def dot_scatter(
        visible: bool = True,
        x_dots: jnp.array = jnp.array([]),
        y_dots: jnp.array = jnp.array([]),
        name_dots: str = 'Observed points',
        showlegend: bool = True
) -> go.Scatter:
    # Adding the dots
    return go.Scatter(
        x=x_dots,
        visible=visible,
        y=y_dots,
        mode="markers",
        name=name_dots,
        marker=dict(color='red', size=8),
        showlegend=showlegend
    )


def uncertainty_area_scatter(
        visible: bool = True,
        x_lines: jnp.array = jnp.array([]),
        y_upper: jnp.array = jnp.array([]),
        y_lower: jnp.array = jnp.array([]),
        name: str = "mean plus/minus standard deviation",
) -> go.Scatter:
    return go.Scatter(
        visible=visible,
        x=jnp.concatenate((x_lines, x_lines[::-1])),  # x, then x reversed
        # upper, then lower reversed
        y=jnp.concatenate((y_upper, y_lower[::-1])),
        fill='toself',
        fillcolor='rgba(189,195,199,0.5)',
        line=dict(color='rgba(200,200,200,0)'),
        hoverinfo="skip",
        showlegend=True,
        name=name,
    )


def add_slider_GPR(figure: go.Figure, parameters):
    figure.data[0].visible = True
    figure.data[1].visible = True

    # Create and add slider
    steps = []
    for i in range(int((len(figure.data) - 1) / 2)):
        step = dict(
            method="update",
            label=f'{parameters[i]: .2f}',
            args=[{
                "visible": [False] * (len(figure.data) - 1) + [True]
            }],
        )
        step["args"][0]["visible"][2 *
                                   i] = True  # Toggle i'th trace to "visible"
        step["args"][0]["visible"][2 * i + 1] = True
        steps.append(step)

    sliders = [dict(
        active=0,
        pad={"t": 50},
        steps=steps,
    )]
    figure.update_layout(sliders=sliders, )
    return figure


def add_slider_to_function(figure: go.Figure, parameters):
    figure.data[0].visible = True

    # Create and add slider
    steps = []
    for i in range(len(figure.data)):
        step = dict(
            method="update",
            label=f'{parameters[i]: .2f}',
            args=[{
                "visible": [False] * len(figure.data)
            }],
        )
        step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        active=0,
        pad={"t": 50},
        steps=steps,
    )]
    figure.update_layout(sliders=sliders, )
    return figure


def plot_GPR(data_x, data_y, model, x, visible=True) -> list:
    mean = model.predict(x)

    std = jnp.sqrt(model._memory['variance'])
    data = []

    for i in range(1, 4):
        data.append(
            uncertainty_area_scatter(
                x_lines=x,
                y_lower=mean - i * std,
                y_upper=mean + i * std,
                name=f"mean plus/minus {i}*standard deviation",
                visible=visible))

    data.append(line_scatter(x_lines=x, y_lines=mean, visible=visible))
    data.append(dot_scatter(x_dots=data_x, y_dots=data_y, visible=visible))
    return data

In [3]:
class SquaredExponentialKernel:
    def __init__(self, sigma_f: float = 1, length: float = 1):
        self.sigma_f = sigma_f
        self.length = length

    def __call__(self, argument_1, argument_2) -> float:
        return float(self.sigma_f * jnp.exp(-(jnp.linalg.norm(argument_1 - argument_2) ** 2) / (2 * self.length ** 2)))


# Helper function to calculate the respective covariance matrices
def cov_matrix(x1, x2, cov_function):
    return jnp.array([[cov_function(a, b) for a in x1] for b in x2])


class GPR:
    def __init__(self, data_x, data_y, covariance_function=SquaredExponentialKernel(), white_noise_sigma: float = 0):
        self.noise = white_noise_sigma
        self.data_x = data_x
        self.data_y = data_y
        self.covariance_function = covariance_function

        # Store the inverse of covariance matrix of input (+ machine epsilon on diagonal) since it is needed for every prediction
        self._inverse_of_covariance_matrix_of_input = jnp.linalg.inv(
            cov_matrix(data_x, data_x, covariance_function) +
            (3e-7 + self.noise) * jnp.identity(len(self.data_x)))

        self._memory = None

    # function to predict output at new input values. Store the mean and covariance matrix in memory.
    def predict(self, at_values):
        k_lower_left = cov_matrix(self.data_x, at_values, self.covariance_function)
        k_lower_right = cov_matrix(at_values, at_values, self.covariance_function)

        # Mean.
        mean_at_values = jnp.dot(k_lower_left,
                                 jnp.dot(self.data_y, self._inverse_of_covariance_matrix_of_input.T).T).flatten()

        # Covariance.
        cov_at_values = k_lower_right - jnp.dot(k_lower_left,
                                                jnp.dot(self._inverse_of_covariance_matrix_of_input, k_lower_left.T))

        # Adding value larger than machine epsilon to ensure positive semi definite
        cov_at_values = cov_at_values + 3e-7 * jnp.ones(jnp.shape(cov_at_values)[0])

        var_at_values = jnp.diag(cov_at_values)

        self._memory = {
            'mean': mean_at_values,
            'covariance_matrix': cov_at_values,
            'variance': var_at_values
        }
        return mean_at_values

In [4]:
x_values = jnp.array([0, 0.3, 1, 3.1, 4.7])

y_values = jnp.array([1, 0, 1.4, 0, -0.9])

x = jnp.arange(-1, 7, 0.1)

In [5]:
model = GPR(x_values, y_values, SquaredExponentialKernel())
model.predict(x)

DeviceArray([ 4.74131870e+00,  4.68346167e+00,  4.52992773e+00,
              4.28164673e+00,  3.94466066e+00,  3.53031325e+00,
              3.05494356e+00,  2.53927827e+00,  2.00740910e+00,
              1.48544598e+00,  9.99980271e-01,  5.76358259e-01,
              2.37036407e-01,  2.08460606e-05, -1.22379646e-01,
             -1.24415830e-01, -7.33258575e-03,  2.20683411e-01,
              5.45146406e-01,  9.46194351e-01,  1.39999259e+00,
              1.88031638e+00,  2.36026430e+00,  2.81392074e+00,
              3.21788573e+00,  3.55252504e+00,  3.80291080e+00,
              3.95935941e+00,  4.01756907e+00,  3.97844481e+00,
              3.84754109e+00,  3.63432932e+00,  3.35124612e+00,
              3.01271892e+00,  2.63413572e+00,  2.23095226e+00,
              1.81786871e+00,  1.40819466e+00,  1.01338220e+00,
              6.42745674e-01,  3.03342104e-01, -2.42002034e-06,
             -2.64539897e-01, -4.89366204e-01, -6.75126791e-01,
             -8.23698878e-01, -9.3786734

In [12]:
x_lines = jnp.arange(-10, 10, 0.1)
kernel = SquaredExponentialKernel(length=5, sigma_f=5)

fig0 = go.FigureWidget(data=[
    line_scatter(
        x_lines=x_lines,
        y_lines=jnp.array([kernel(x, 0) for x in x_lines]),
    )
])

fig0 = update_layout_of_graph(fig0, title='Squared exponential kernel')
fig0

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'line': {'color': 'blue', 'width': 2},
              'name': 'Predicted function',
              'showlegend': True,
              'type': 'scatter',
              'uid': '6a8d2418-b55b-4489-9651-4978a4240756',
              'visible': True,
              'x': array([-1.0000000e+01, -9.8999996e+00, -9.7999992e+00, -9.6999989e+00,
                          -9.5999985e+00, -9.4999981e+00, -9.3999977e+00, -9.2999973e+00,
                          -9.1999969e+00, -9.0999966e+00, -8.9999962e+00, -8.8999958e+00,
                          -8.7999954e+00, -8.6999950e+00, -8.5999947e+00, -8.4999943e+00,
                          -8.3999939e+00, -8.2999935e+00, -8.1999931e+00, -8.0999928e+00,
                          -7.9999924e+00, -7.8999920e+00, -7.7999916e+00, -7.6999912e+00,
                          -7.5999908e+00, -7.4999905e+00, -7.3999901e+00, -7.2999897e+00,
                          -7.1999893e+00, -7.0999889e+00, -6.9999886e+00, -6.8999882e+00,
                          -6.7999878e+00, -6.6999874e+00, -6.5999870e+00, -6.4999866e+00,
                          -6.3999863e+00, -6.2999859e+00, -6.1999855e+00, -6.0999851e+00,
                          -5.9999847e+00, -5.8999844e+00, -5.7999840e+00, -5.6999836e+00,
                          -5.5999832e+00, -5.4999828e+00, -5.3999825e+00, -5.2999821e+00,
                          -5.1999817e+00, -5.0999813e+00, -4.9999809e+00, -4.8999805e+00,
                          -4.7999802e+00, -4.6999798e+00, -4.5999794e+00, -4.4999790e+00,
                          -4.3999786e+00, -4.2999783e+00, -4.1999779e+00, -4.0999775e+00,
                          -3.9999771e+00, -3.8999767e+00, -3.7999763e+00, -3.6999760e+00,
                          -3.5999756e+00, -3.4999752e+00, -3.3999748e+00, -3.2999744e+00,
                          -3.1999741e+00, -3.0999737e+00, -2.9999733e+00, -2.8999729e+00,
                          -2.7999725e+00, -2.6999722e+00, -2.5999718e+00, -2.4999714e+00,
                          -2.3999710e+00, -2.2999706e+00, -2.1999702e+00, -2.0999699e+00,
                          -1.9999695e+00, -1.8999691e+00, -1.7999687e+00, -1.6999683e+00,
                          -1.5999680e+00, -1.4999676e+00, -1.3999672e+00, -1.2999668e+00,
                          -1.1999664e+00, -1.0999660e+00, -9.9996567e-01, -8.9996529e-01,
                          -7.9996490e-01, -6.9996452e-01, -5.9996414e-01, -4.9996376e-01,
                          -3.9996338e-01, -2.9996300e-01, -1.9996262e-01, -9.9962234e-02,
                           3.8146973e-05,  1.0003853e-01,  2.0003891e-01,  3.0003929e-01,
                           4.0003967e-01,  5.0004005e-01,  6.0004044e-01,  7.0004082e-01,
                           8.0004120e-01,  9.0004158e-01,  1.0000420e+00,  1.1000423e+00,
                           1.2000427e+00,  1.3000431e+00,  1.4000435e+00,  1.5000439e+00,
                           1.6000443e+00,  1.7000446e+00,  1.8000450e+00,  1.9000454e+00,
                           2.0000458e+00,  2.1000462e+00,  2.2000465e+00,  2.3000469e+00,
                           2.4000473e+00,  2.5000477e+00,  2.6000481e+00,  2.7000484e+00,
                           2.8000488e+00,  2.9000492e+00,  3.0000496e+00,  3.1000500e+00,
                           3.2000504e+00,  3.3000507e+00,  3.4000511e+00,  3.5000515e+00,
                           3.6000519e+00,  3.7000523e+00,  3.8000526e+00,  3.9000530e+00,
                           4.0000534e+00,  4.1000538e+00,  4.2000542e+00,  4.3000546e+00,
                           4.4000549e+00,  4.5000553e+00,  4.6000557e+00,  4.7000561e+00,
                           4.8000565e+00,  4.9000568e+00,  5.0000572e+00,  5.1000576e+00,
                           5.2000580e+00,  5.3000584e+00,  5.4000587e+00,  5.5000591e+00,
                           5.6000595e+00,  5.7000599e+00,  5.8000603e+00,  5.9000607e+00,
                           6.0000610e+00,  6.1000614e+00,  6.2000618e+00,  6.3000622e+00,
                           6.4000626e+0

In [22]:
model = GPR(x_values, y_values, SquaredExponentialKernel(sigma_f=0.5, length=.5))
data = plot_GPR(data_x=x_values, data_y=y_values, x=x, model=model)
fig4 = go.Figure(data=data)
fig4 = update_layout_of_graph(fig=fig4, title='GPR with length 1, sigma 0 and noise 0')

fig4.show()